In [1]:

from omop_semantics.schema.generated_models.omop_semantic_registry import RegistryFragment
from omop_semantics.runtime.resolver import OmopSemanticResolver, OmopRegistryRuntime, OmopTemplateRuntime, OmopSemanticEngine, merge_registry_fragments
from pathlib import Path
from omop_semantics.runtime.instance_loader import load_registry_fragment, merge_instance_files, load_profiles
from linkml_runtime.loaders import yaml_loader

instance_base = Path('../omop_semantics/schema/instances/')
profile_base = Path('../omop_semantics/schema/configuration/profiles')


In [2]:
profiles = load_profiles(instance_base / "profiles.yaml")

registry_dict = merge_instance_files(
    paths = [instance_base / "demographic.yaml"],
    profiles = profiles
)

In [3]:
fragment = yaml_loader.load(registry_dict, target_class=RegistryFragment)

In [4]:
def load_symbol_module(path: Path) -> dict[str, dict]:
    raw = yaml_loader.load_as_dict(str(path))

    if not isinstance(raw, dict):
        raise TypeError(f"Expected mapping at top level in {path}, got {type(raw)}")

    # Filter out schema-level keys
    skip = {"id", "name", "description", "imports", "prefixes", "default_prefix", "default_range"}
    
    return {
        k: v for k, v in raw.items()
        if k not in skip and isinstance(v, dict)
    }

In [5]:
staging_symbols = load_symbol_module(profile_base / 'omop_staging.yaml')

In [6]:
engine = OmopSemanticEngine(
    registry_fragment=fragment,
    profile_objects=staging_symbols,   # or merge with profile symbols too
)

In [7]:
#

In [8]:
engine.registry_runtime

Template,Role,CDM Profile,Entity Concept,Value Concept
Country of birth,demographic,name='observation_simple' cdm_table='observation' concept_slot='observation_concept_id' value_slot=None,Group: Country of BirthAnchors: 4155450 (Country of birth),None
Language spoken,demographic,name='observation_coded' cdm_table='observation' concept_slot='observation_concept_id' value_slot='value_as_concept_id',"Concept: Language Spoken (concept_id=4052785, label=Language spoken)",Group: Language Spoken ValuesAnchors: 4182347 (World languages)
Postcode,demographic,name='observation_string' cdm_table='observation' concept_slot='observation_concept_id' value_slot='value_as_string',"Concept: Postcode (concept_id=4083591, label=Postcode)",None


In [9]:
engine.registry_runtime.compile_all(role="demographic")
engine.registry_runtime.to_compiled_html(role="demographic")

Template,CDM Profile,Entity Concept IDs,Value Concept IDs
Country of birth,name='observation_simple' cdm_table='observation' concept_slot='observation_concept_id' value_slot=None,4155450,
Language spoken,name='observation_coded' cdm_table='observation' concept_slot='observation_concept_id' value_slot='value_as_concept_id',4052785,4182347
Postcode,name='observation_string' cdm_table='observation' concept_slot='observation_concept_id' value_slot='value_as_string',4083591,


In [10]:
engine.profile_runtime